In [ ]:
#Extracting tickers in S&P500 that are founded before 2021.

import pandas as pd

wiki = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

tables = pd.read_html(wiki)
SP500 = tables[0]

SP500_2021 = SP500.loc[SP500['Founded'].str[:4].astype(int) < 2021, 'Symbol']
tickers2021 = [s.replace('.', '-') for s in SP500_2021]

In [ ]:
#Download data from yfinance for selected tickers of the previous part.

import yfinance as yf

Data = {}
for x in tickers2021:
  Data[x] = yf.download(x, period='1y', interval='1d', progress=False)

In [ ]:
#FIRST METRIC: This metric gives more weight to stocks with significant price increases and higher trading volumes.

import math
import numpy as np

Scores1 = []

for x in tickers2021:
  Price_change = (Data[x]['Close'][-1] - Data[x]['Close'][0]) / Data[x]['Close'][0]
  Ave_vol = sum(Data[x]['Volume'])/len(Data[x]['Volume'])
  Adj_Score = Price_change * math.log(Ave_vol)
  Scores1.append([x, Adj_Score])

Scores1.sort(key=lambda x: x[1], reverse = True)
Top10_1 = Scores1[:10]

In [ ]:
#SECOND METRIC: Linear extrapolation

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

slope_parameter = []

for x in tickers2021:
  Temp = Data[x]
  X_train = np.array([i for i in range(len(Temp))]).reshape(-1, 1)
  y_train = Temp['Close'].values.reshape(-1, 1)
  model = LinearRegression()
  model.fit(X_train, y_train)
  slope_parameter.append([x, model.coef_[0][0]])

for x in slope_parameter:
  temp = x[1]*30/Data[x[0]]['Close'][-1]
  x[1] = temp
slope_parameter.sort(key=lambda x: x[1], reverse = True)
Top10_2 = slope_parameter[:10]

In [ ]:
#THIRD SCORE: ARIMA

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import itertools
import statsmodels.api as sm

Scores3 =[]

for x in tickers2021:
  x = slope_parameter[i][0]
  Close = Data[x]['Close']
  p = q = range(0, 5)
  d = range(0, 3)
  pdq = list(itertools.product(p, d, q))
  results = []
  for param in pdq:
    try:
        model = sm.tsa.ARIMA(Close, order=param)
        model_fit = model.fit()
        results.append([param, model_fit.aic])
    except:
        continue
  results.sort(key=lambda x: x[1])
  best_params = results[0][0]
  model = sm.tsa.ARIMA(Close, order=best_params)
  model_fit = model.fit()
  forecast = model_fit.forecast(steps= 30)
  Scores3.append([x, 100*(forecast[280] - Close[-1])/Close[-1], best_params])

Scores3.sort(key=lambda x: x[1], reverse = True)
Top10_3 = Scores3[:10]
print(Top10_3)

In [ ]:
#Pie plot for Top10 of Score 1

import matplotlib.pyplot as plt

labels = [x[0] for x in Top10_1]
sizes = [x[1] for x in Top10_1]

fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)

ax.axis('equal')
plt.show()

In [ ]:
#Pie plot for Top10 of Score 2

labels = [x[0] for x in Top10_2]
sizes = [x[1] for x in Top10_2]

fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)

ax.axis('equal')
plt.show()

In [ ]:
#Pie plot for Top10 of Score 3

labels = [x[0] for x in Top10_3]
sizes = [x[1] for x in Top10_3]

fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)

ax.axis('equal')
plt.show()